# Multi-Regime Distribution Validation (Real Data)

## Objective
Test whether the diffusion model can learn **different probability distributions** for **different market conditions** using **real stock data patterns**.

## Experiment Design

We extract real patterns from `data_1s/` and group them by their subsequent 30-second returns:

| Regime | Description | Target Distribution |
|--------|-------------|--------------------|
| **Bullish** | Patterns preceding positive moves | ~80% positive returns |
| **Bearish** | Patterns preceding negative moves | ~80% negative returns |
| **Neutral** | Patterns with small subsequent moves | ~50/50, small magnitude |

## Key Differences from Synthetic Data
- Return magnitudes are realistic (~0.2% std for 30s horizon)
- Condition patterns extracted from actual market data
- Feature distributions match real trading dynamics

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Dict, Tuple, List, Optional
import gc

from src.models.model import StockDiffusionModel, ModelConfig
from src.data.loader import StockDataLoader, DataConfig
from src.data.preprocessing import DataPreprocessor, PreprocessConfig

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Device setup
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

## 1. Extract Real Market Patterns

We load actual stock data and categorize sequences by their outcomes.

In [ ]:
# Configuration
SEQ_LENGTH = 120   # 120 seconds of history
HORIZON = 30       # 30 seconds prediction horizon
N_FEATURES = 4     # log_return, volatility_5s, spread, volume_zscore
FEATURE_NAMES = ['log_return', 'volatility_5s', 'spread', 'volume_zscore']

# Data paths
data_dir = project_root / 'data_1s'
print(f"Data directory: {data_dir}")
print(f"Sequence length: {SEQ_LENGTH}s")
print(f"Prediction horizon: {HORIZON}s")

In [ ]:
def extract_real_patterns(
    data_dir: Path,
    n_dates: int = 15,
    n_symbols_per_date: int = 30,
    seq_length: int = 120,
    horizon: int = 30,
    sample_stride: int = 30,
    verbose: bool = True
) -> Tuple[Dict[str, np.ndarray], Dict[str, np.ndarray], float]:
    """
    Extract real market patterns and categorize by outcome.
    
    Returns:
        conditions: Dict mapping regime -> array of condition sequences
        returns: Dict mapping regime -> array of target returns
        threshold: Return threshold used for categorization
    """
    config = DataConfig(data_dir=data_dir)
    loader = StockDataLoader(config)
    preprocessor = DataPreprocessor(PreprocessConfig())
    
    dates = loader.get_available_dates()[:n_dates]
    if verbose:
        print(f"Processing {len(dates)} dates...")
    
    all_sequences = []  # List of (condition, return)
    
    for date in tqdm(dates, disable=not verbose, desc="Extracting patterns"):
        symbols = loader.get_available_symbols(date)[:n_symbols_per_date]
        
        for symbol in symbols:
            df = loader.load_symbol_day(symbol, date, filter_market_hours=True)
            if df is None or len(df) < seq_length + horizon + 10:
                continue
            
            try:
                processed, valid = preprocessor.process_day(df)
                
                # Extract sequences
                for i in range(0, len(processed) - seq_length - horizon, sample_stride):
                    if not valid[i:i + seq_length + horizon].all():
                        continue
                    
                    # Condition: feature sequence
                    cond = processed[FEATURE_NAMES].iloc[i:i + seq_length].values
                    
                    # Target: forward return
                    p_now = df['close'].iloc[i + seq_length]
                    p_future = df['close'].iloc[i + seq_length + horizon]
                    ret = (p_future / p_now) - 1
                    
                    if np.isfinite(cond).all() and np.isfinite(ret):
                        all_sequences.append((cond.astype(np.float32), float(ret)))
            except Exception as e:
                continue
        
        gc.collect()
    
    if verbose:
        print(f"\nExtracted {len(all_sequences):,} sequences")
    
    # Analyze return distribution
    returns_all = np.array([r for _, r in all_sequences])
    
    if verbose:
        print(f"\nReturn distribution:")
        print(f"  Mean: {returns_all.mean()*100:.4f}%")
        print(f"  Std:  {returns_all.std()*100:.4f}%")
        print(f"  5th percentile: {np.percentile(returns_all, 5)*100:.4f}%")
        print(f"  95th percentile: {np.percentile(returns_all, 95)*100:.4f}%")
    
    # Categorize by return magnitude (top/bottom ~20%)
    threshold = np.percentile(np.abs(returns_all), 60)
    
    bullish = [(c, r) for c, r in all_sequences if r > threshold]
    bearish = [(c, r) for c, r in all_sequences if r < -threshold]
    neutral = [(c, r) for c, r in all_sequences if abs(r) <= threshold]
    
    if verbose:
        print(f"\nCategorization (threshold: ±{threshold*100:.4f}%):")
        print(f"  Bullish: {len(bullish):,} samples, mean={np.mean([r for _,r in bullish])*100:.4f}%")
        print(f"  Bearish: {len(bearish):,} samples, mean={np.mean([r for _,r in bearish])*100:.4f}%")
        print(f"  Neutral: {len(neutral):,} samples, mean={np.mean([r for _,r in neutral])*100:.4f}%")
    
    # Return organized data
    conditions = {
        'bullish': np.stack([c for c, _ in bullish]),
        'bearish': np.stack([c for c, _ in bearish]),
        'neutral': np.stack([c for c, _ in neutral])
    }
    
    returns = {
        'bullish': np.array([r for _, r in bullish], dtype=np.float32),
        'bearish': np.array([r for _, r in bearish], dtype=np.float32),
        'neutral': np.array([r for _, r in neutral], dtype=np.float32)
    }
    
    return conditions, returns, threshold

In [ ]:
# Extract patterns from real data
conditions, returns, threshold = extract_real_patterns(
    data_dir=data_dir,
    n_dates=20,            # Use 20 days
    n_symbols_per_date=40, # 40 stocks per day
    seq_length=SEQ_LENGTH,
    horizon=HORIZON,
    sample_stride=20,      # Sample every 20 seconds
    verbose=True
)

In [ ]:
# Visualize the extracted patterns
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

regimes = ['bullish', 'bearish', 'neutral']
colors = {'bullish': 'green', 'bearish': 'red', 'neutral': 'gray'}
titles = ['Bullish (r > threshold)', 'Bearish (r < -threshold)', 'Neutral (small |r|)']

for idx, (regime, title) in enumerate(zip(regimes, titles)):
    conds = conditions[regime]
    rets = returns[regime]
    color = colors[regime]
    
    # Top row: Average condition pattern
    ax = axes[0, idx]
    mean_cond = conds.mean(axis=0)
    for i, feat in enumerate(FEATURE_NAMES):
        ax.plot(mean_cond[:, i], label=feat, alpha=0.8)
    ax.set_title(f'{title}\nAverage Condition (n={len(conds):,})')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Feature Value')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    
    # Bottom row: Return distribution
    ax = axes[1, idx]
    ax.hist(rets * 100, bins=50, color=color, alpha=0.7, edgecolor='black')
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
    ax.axvline(x=rets.mean() * 100, color='blue', linestyle='-', linewidth=2, 
               label=f'Mean: {rets.mean()*100:.3f}%')
    ax.set_title(f'Return Distribution')
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Count')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nReturn threshold: ±{threshold*100:.4f}%")

## 2. Create Regime-Specific Datasets

In [ ]:
class RealRegimeDataset(Dataset):
    """
    Dataset for a specific market regime using real data patterns.
    
    Uses a representative condition pattern for each regime,
    paired with the actual return distribution from that regime.
    """
    
    def __init__(
        self,
        regime: str,
        regime_conditions: np.ndarray,
        regime_returns: np.ndarray,
        n_samples: int = 2000,
        use_mean_pattern: bool = True
    ):
        self.regime = regime
        self.n_samples = n_samples
        
        if use_mean_pattern:
            # Use mean pattern as representative condition
            self.condition = regime_conditions.mean(axis=0).astype(np.float32)
        else:
            # Use a random pattern
            idx = np.random.randint(len(regime_conditions))
            self.condition = regime_conditions[idx].astype(np.float32)
        
        # Sample targets from real distribution (with replacement)
        indices = np.random.choice(len(regime_returns), size=n_samples, replace=True)
        self.targets = regime_returns[indices].astype(np.float32)
        
    def __len__(self) -> int:
        return self.n_samples
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        return {
            'input': torch.from_numpy(self.condition.copy()),
            'target': torch.tensor([self.targets[idx]], dtype=torch.float32)
        }
    
    def get_condition_tensor(self) -> torch.Tensor:
        return torch.from_numpy(self.condition.copy())
    
    def get_stats(self) -> Dict:
        return {
            'regime': self.regime,
            'n_samples': self.n_samples,
            'mean': self.targets.mean(),
            'std': self.targets.std(),
            'pct_positive': (self.targets > 0).mean() * 100
        }

In [ ]:
# Create datasets for each regime
N_SAMPLES_PER_REGIME = 3000

datasets = {}
for regime in ['bullish', 'bearish', 'neutral']:
    datasets[regime] = RealRegimeDataset(
        regime=regime,
        regime_conditions=conditions[regime],
        regime_returns=returns[regime],
        n_samples=N_SAMPLES_PER_REGIME,
        use_mean_pattern=True
    )

# Print statistics
print("Dataset Statistics (Real Data):")
print("=" * 70)
print(f"{'Regime':<12} {'Samples':<10} {'Mean':>12} {'Std':>10} {'%Positive':>12}")
print("-" * 70)
for regime, ds in datasets.items():
    stats = ds.get_stats()
    print(f"{stats['regime']:<12} {stats['n_samples']:<10} {stats['mean']*100:>+11.4f}% {stats['std']*100:>9.4f}% {stats['pct_positive']:>11.1f}%")

In [ ]:
# Compare condition patterns between regimes
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for i, feat in enumerate(FEATURE_NAMES):
    ax = axes[i // 2, i % 2]
    for regime in ['bullish', 'bearish', 'neutral']:
        pattern = datasets[regime].condition[:, i]
        ax.plot(pattern, color=colors[regime], label=regime.capitalize(), alpha=0.8, linewidth=2)
    ax.set_title(f'Feature: {feat}')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Value')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Condition Patterns by Regime (Real Data)', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Normalize and Create Model

In [ ]:
# Compute normalization statistics
all_conditions = np.concatenate([conditions[r] for r in regimes], axis=0)
norm_mean = all_conditions.reshape(-1, N_FEATURES).mean(axis=0)
norm_std = all_conditions.reshape(-1, N_FEATURES).std(axis=0) + 1e-8

print("Normalization statistics:")
for i, feat in enumerate(FEATURE_NAMES):
    print(f"  {feat}: mean={norm_mean[i]:.6f}, std={norm_std[i]:.6f}")

In [ ]:
class NormalizedDataset(Dataset):
    """Wrapper that normalizes conditions."""
    
    def __init__(self, base_dataset: RealRegimeDataset, mean: np.ndarray, std: np.ndarray):
        self.base = base_dataset
        self.mean = mean.astype(np.float32)
        self.std = std.astype(np.float32)
        self.normalized_condition = (self.base.condition - self.mean) / self.std
        
    def __len__(self):
        return len(self.base)
    
    def __getitem__(self, idx):
        return {
            'input': torch.from_numpy(self.normalized_condition.copy()),
            'target': torch.tensor([self.base.targets[idx]], dtype=torch.float32)
        }
    
    def get_condition_tensor(self):
        return torch.from_numpy(self.normalized_condition.copy())
    
    @property
    def regime(self):
        return self.base.regime
    
    @property
    def targets(self):
        return self.base.targets


# Create normalized datasets
norm_datasets = {
    regime: NormalizedDataset(ds, norm_mean, norm_std)
    for regime, ds in datasets.items()
}

print("Created normalized datasets")

In [ ]:
# Combine for training
combined_dataset = ConcatDataset(list(norm_datasets.values()))
print(f"Combined dataset size: {len(combined_dataset)} samples")

# Create model
model_config = ModelConfig(
    encoder_type="transformer",
    input_features=N_FEATURES,
    seq_length=SEQ_LENGTH,
    encoder_dim=64,
    encoder_layers=3,
    encoder_heads=4,
    diffusion_steps=500,
    noise_schedule="cosine",
    prediction_type="epsilon",
    denoiser_type="mlp",
    denoising_hidden_dim=128,
    denoising_blocks=4,
    time_embedding_dim=64,
    dropout=0.1
)

model = StockDiffusionModel(model_config).to(device)
print(f"Model parameters: {model.get_num_params():,}")

## 4. Train the Model

In [ ]:
def train_model(
    model: nn.Module,
    dataset: Dataset,
    epochs: int = 200,
    batch_size: int = 128,
    learning_rate: float = 3e-4,
    device: torch.device = device
) -> List[float]:
    """Train the diffusion model."""
    
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=learning_rate/20)
    
    losses = []
    model.train()
    
    pbar = tqdm(range(epochs), desc="Training")
    for epoch in pbar:
        epoch_loss = 0.0
        n_batches = 0
        
        for batch in dataloader:
            x_seq = batch['input'].to(device)
            target = batch['target'].to(device)
            
            optimizer.zero_grad()
            outputs = model(x_seq, target)
            loss = outputs['loss']
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
            n_batches += 1
        
        scheduler.step()
        avg_loss = epoch_loss / n_batches
        losses.append(avg_loss)
        pbar.set_postfix({'loss': f'{avg_loss:.6f}', 'lr': f'{scheduler.get_last_lr()[0]:.2e}'})
    
    return losses

In [ ]:
# Train
EPOCHS = 250
BATCH_SIZE = 128
LEARNING_RATE = 3e-4

print(f"Training: {EPOCHS} epochs, batch_size={BATCH_SIZE}, lr={LEARNING_RATE}")
print(f"Total samples: {len(combined_dataset)}\n")

losses = train_model(model, combined_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE)

plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print(f"\nFinal loss: {losses[-1]:.6f}")

## 5. Sample from Each Regime

In [ ]:
@torch.no_grad()
def sample_regime(
    model: nn.Module,
    condition: torch.Tensor,
    n_samples: int = 500,
    use_ddim: bool = True,
    ddim_steps: int = 50,
    device: torch.device = device
) -> np.ndarray:
    model.eval()
    condition = condition.unsqueeze(0).to(device).expand(n_samples, -1, -1)
    samples = model.sample(condition, num_samples=1, use_ddim=use_ddim, ddim_steps=ddim_steps)
    return samples.cpu().numpy().flatten()

In [ ]:
# Sample from each regime
N_INFERENCE_SAMPLES = 500

print(f"Sampling {N_INFERENCE_SAMPLES} predictions per regime...\n")

predictions = {}
for regime, ds in norm_datasets.items():
    condition = ds.get_condition_tensor()
    preds = sample_regime(model, condition, n_samples=N_INFERENCE_SAMPLES)
    predictions[regime] = preds
    
    print(f"{regime.capitalize()}:")
    print(f"  GT:   mean={ds.targets.mean()*100:+.4f}%, std={ds.targets.std()*100:.4f}%, pos={(ds.targets>0).mean()*100:.1f}%")
    print(f"  Pred: mean={preds.mean()*100:+.4f}%, std={preds.std()*100:.4f}%, pos={(preds>0).mean()*100:.1f}%")
    print()

## 6. Visualize Results

In [ ]:
# Compare GT vs predictions
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

for idx, regime in enumerate(regimes):
    ds = norm_datasets[regime]
    preds = predictions[regime]
    color = colors[regime]
    
    # Top: Ground truth
    ax = axes[0, idx]
    ax.hist(ds.targets * 100, bins=50, color=color, alpha=0.7, edgecolor='black', density=True)
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
    ax.axvline(x=ds.targets.mean() * 100, color='blue', linestyle='-', linewidth=2, 
               label=f'Mean: {ds.targets.mean()*100:+.4f}%')
    ax.set_title(f'{regime.capitalize()}\nGround Truth', fontsize=12)
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Density')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    
    # Bottom: Predictions
    ax = axes[1, idx]
    ax.hist(preds * 100, bins=50, color=color, alpha=0.7, edgecolor='black', density=True)
    ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
    ax.axvline(x=preds.mean() * 100, color='blue', linestyle='-', linewidth=2, 
               label=f'Pred: {preds.mean()*100:+.4f}%')
    ax.axvline(x=ds.targets.mean() * 100, color='red', linestyle=':', linewidth=2, 
               label=f'GT: {ds.targets.mean()*100:+.4f}%')
    ax.set_title(f'{regime.capitalize()}\nPredictions (n={N_INFERENCE_SAMPLES})', fontsize=12)
    ax.set_xlabel('Return (%)')
    ax.set_ylabel('Density')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(project_root / 'outputs' / 'real_regime_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("Saved to outputs/real_regime_comparison.png")

In [ ]:
# Overlay all regimes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
for regime in regimes:
    ds = norm_datasets[regime]
    ax.hist(ds.targets * 100, bins=50, color=colors[regime], alpha=0.5, 
            label=f"{regime.capitalize()} (μ={ds.targets.mean()*100:+.3f}%)", density=True)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
ax.set_title('Ground Truth', fontsize=12)
ax.set_xlabel('Return (%)')
ax.set_ylabel('Density')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
for regime in regimes:
    preds = predictions[regime]
    ax.hist(preds * 100, bins=50, color=colors[regime], alpha=0.5, 
            label=f"{regime.capitalize()} (μ={preds.mean()*100:+.3f}%)", density=True)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1.5)
ax.set_title('Predictions', fontsize=12)
ax.set_xlabel('Return (%)')
ax.set_ylabel('Density')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Analysis and Verdict

In [ ]:
print("=" * 80)
print("QUANTITATIVE ANALYSIS (Real Data)")
print("=" * 80)

for regime in regimes:
    ds = norm_datasets[regime]
    preds = predictions[regime]
    
    print(f"\n{regime.upper()}")
    print("-" * 60)
    print(f"  {'Metric':<25} {'Ground Truth':>15} {'Prediction':>15}")
    print(f"  {'-'*55}")
    print(f"  {'Mean Return':<25} {ds.targets.mean()*100:>+14.4f}% {preds.mean()*100:>+14.4f}%")
    print(f"  {'Std Dev':<25} {ds.targets.std()*100:>14.4f}% {preds.std()*100:>14.4f}%")
    print(f"  {'% Positive':<25} {(ds.targets>0).mean()*100:>14.1f}% {(preds>0).mean()*100:>14.1f}%")

In [ ]:
# Final verdict
print("\n" + "=" * 80)
print("FINAL VERDICT")
print("=" * 80)

bullish_preds = predictions['bullish']
bearish_preds = predictions['bearish']
neutral_preds = predictions['neutral']

# Criteria
bullish_correct = bullish_preds.mean() > bearish_preds.mean() and (bullish_preds > 0).mean() > (bearish_preds > 0).mean()
bearish_correct = bearish_preds.mean() < bullish_preds.mean() and (bearish_preds < 0).mean() > (bullish_preds < 0).mean()
neutral_correct = bearish_preds.mean() < neutral_preds.mean() < bullish_preds.mean()
means_different = bullish_preds.mean() - bearish_preds.mean() > 0.0005

bullish_dir = (bullish_preds.mean() > 0) == (norm_datasets['bullish'].targets.mean() > 0)
bearish_dir = (bearish_preds.mean() < 0) == (norm_datasets['bearish'].targets.mean() < 0)

print(f"\nCriteria Check:")
print(f"  [{'✓' if bullish_correct else '✗'}] Bullish > Bearish")
print(f"  [{'✓' if bearish_correct else '✗'}] Bearish < Bullish")
print(f"  [{'✓' if neutral_correct else '✗'}] Neutral in between")
print(f"  [{'✓' if means_different else '✗'}] Distributions distinct")
print(f"  [{'✓' if bullish_dir else '✗'}] Bullish direction correct")
print(f"  [{'✓' if bearish_dir else '✗'}] Bearish direction correct")

all_pass = bullish_correct and bearish_correct and means_different

print("\n" + "-" * 60)
if all_pass:
    print("\n🎉 [PASS] Model learned conditional distributions from real data!")
    print(f"\n  Bullish: {bullish_preds.mean()*100:+.4f}%")
    print(f"  Bearish: {bearish_preds.mean()*100:+.4f}%")
    print(f"  Neutral: {neutral_preds.mean()*100:+.4f}%")
elif bullish_dir and bearish_dir:
    print("\n⚠️  [PARTIAL] Model learned direction correctly")
else:
    print("\n❌ [FAIL] Model did not learn distinct distributions")

print("\n" + "=" * 80)

## 8. Interactive Sampling

In [ ]:
def sample_and_visualize(regime: str, n_samples: int = 500):
    """Sample from a regime and visualize."""
    if regime not in norm_datasets:
        print(f"Unknown regime: {regime}")
        return
    
    ds = norm_datasets[regime]
    color = colors[regime]
    
    print(f"Sampling {n_samples} predictions for {regime}...")
    preds = sample_regime(model, ds.get_condition_tensor(), n_samples=n_samples)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    ax = axes[0]
    ax.hist(ds.targets * 100, bins=40, alpha=0.5, label='Ground Truth', density=True)
    ax.hist(preds * 100, bins=40, alpha=0.5, label='Predictions', density=True, color=color)
    ax.axvline(x=0, color='black', linestyle='--')
    ax.set_title(f'{regime.capitalize()} - Distribution')
    ax.set_xlabel('Return (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    ax = axes[1]
    ax.scatter(range(len(preds)), preds * 100, alpha=0.5, s=10, c=color)
    ax.axhline(y=0, color='black', linestyle='--')
    ax.axhline(y=ds.targets.mean() * 100, color='red', linewidth=2, label=f'GT: {ds.targets.mean()*100:+.4f}%')
    ax.axhline(y=preds.mean() * 100, color='blue', linewidth=2, label=f'Pred: {preds.mean()*100:+.4f}%')
    ax.set_title(f'{regime.capitalize()} - Samples')
    ax.set_xlabel('Index')
    ax.set_ylabel('Return (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"GT:   mean={ds.targets.mean()*100:+.4f}%, pos={(ds.targets>0).mean()*100:.1f}%")
    print(f"Pred: mean={preds.mean()*100:+.4f}%, pos={(preds>0).mean()*100:.1f}%")
    return preds

In [ ]:
_ = sample_and_visualize('bullish', 500)

In [ ]:
_ = sample_and_visualize('bearish', 500)

In [ ]:
_ = sample_and_visualize('neutral', 500)

## 9. Save Results

In [ ]:
import json

results = {
    'config': {
        'seq_length': SEQ_LENGTH,
        'horizon': HORIZON,
        'n_samples_per_regime': N_SAMPLES_PER_REGIME,
        'epochs': EPOCHS,
        'return_threshold': float(threshold),
        'data_source': 'real_stock_data'
    },
    'results': {
        regime: {
            'gt_mean': float(norm_datasets[regime].targets.mean()),
            'gt_std': float(norm_datasets[regime].targets.std()),
            'gt_pct_positive': float((norm_datasets[regime].targets > 0).mean() * 100),
            'pred_mean': float(predictions[regime].mean()),
            'pred_std': float(predictions[regime].std()),
            'pred_pct_positive': float((predictions[regime] > 0).mean() * 100)
        }
        for regime in regimes
    },
    'final_loss': float(losses[-1]),
    'verdict': {'all_pass': bool(all_pass)}
}

output_path = project_root / 'outputs' / 'real_regime_validation_results.json'
output_path.parent.mkdir(parents=True, exist_ok=True)
with open(output_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Saved to: {output_path}")